In [2]:
!pip install pywikibot

   ---------------------------------------- 0.0/723.8 kB ? eta -:--:--
   -------------- ------------------------- 262.1/723.8 kB ? eta -:--:--
   ---------------------------------------- 723.8/723.8 kB 2.5 MB/s eta 0:00:00


In [14]:
import requests
from bs4 import BeautifulSoup

# Step 1: Fetch the edit page to get the sectok
edit_url = "http://192.168.1.247/psyc_jobs/dokuwiki/doku.php?id=wiki:test&do=edit"
response = requests.get(edit_url)

# Step 2: Parse the sectok from the HTML
soup = BeautifulSoup(response.text, 'html.parser')
sectok = soup.find('input', {'name': 'sectok'})['value']

# Step 3: Prepare and send the POST request to edit the page
edit_data = {
    'sectok': sectok,
    'id': 'wiki:test2',
    'rev': '0',
    'date': '1724816599',
    'prefix': '.',
    'suffix': '',
    'changecheck': 'a119e534072584a0ea88cdea4664aecd',
    'target': 'section',
    'wikitext': 'New Content HereNew Content HereNew Content HereNew Content Here',  # Replace with your content
    'do[save]': '1'
}

response = requests.post(edit_url, data=edit_data)

# Step 4: Check if the edit was successful
if response.status_code == 200:
    print("Page edited successfully!")
else:
    print("Failed to edit the page.")


Page edited successfully!


In [17]:
import requests
from bs4 import BeautifulSoup

def edit_page(career, content):
    # Step 1: Fetch the edit page to get the sectok
    edit_url = f"http://192.168.1.247/psyc_jobs/dokuwiki/doku.php?id=wiki:{career}&do=edit"
    response = requests.get(edit_url)

    # Debug: Print response status and content snippet
    print(f"Fetch status code: {response.status_code}")
    print(f"Response content (snippet): {response.text[:500]}")

    # Step 2: Parse the sectok from the HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    sectok = soup.find('input', {'name': 'sectok'})
    if sectok:
        sectok = sectok['value']
        print(f"Extracted sectok: {sectok}")
    else:
        print(f"Failed to extract sectok for page: {career}")
        return

    # Step 3: Prepare and send the POST request to edit the page
    edit_data = {
        'sectok': sectok,
        'id': f'wiki:{career}',
        'rev': '0',
        'date': '1724816599',
        'prefix': '.',
        'suffix': '',
        'changecheck': 'a119e534072584a0ea88cdea4664aecd',
        'target': 'section',
        'wikitext': content,  # Replace with your content
        'do[save]': '1'
    }

    response = requests.post(edit_url, data=edit_data)

    # Debug: Print response status and content snippet
    print(f"Post status code: {response.status_code}")
    print(f"Post response content (snippet): {response.text[:500]}")

    # Step 4: Check if the edit was successful
    if response.status_code == 200:
        print(f"Page '{career}' edited successfully!")
    else:
        print(f"Failed to edit the page '{career}'.")

# Example usage
career = 'Big_Career'
content = 'New Content HereNew Content HereNew Content HereNew Content Here'
edit_page(career, content)


Fetch status code: 200
Response content (snippet): <!DOCTYPE html>
<html lang="en" dir="ltr" class="no-js">
<head>
    <meta charset="utf-8" />
    <title>✎ wiki:big_career [DokuWiki]</title>
    <script>(function(H){H.className=H.className.replace(/\bno-js\b/,'js')})(document.documentElement)</script>
    <meta name="generator" content="DokuWiki"/>
<meta name="theme-color" content="#008800"/>
<meta name="robots" content="noindex,nofollow"/>
<link rel="search" type="application/opensearchdescription+xml" href="/psyc_jobs/dokuwiki/lib/exe/opensea
Extracted sectok: 
Post status code: 200
Post response content (snippet): <!DOCTYPE html>
<html lang="en" dir="ltr" class="no-js">
<head>
    <meta charset="utf-8" />
    <title>wiki:big_career [DokuWiki]</title>
    <script>(function(H){H.className=H.className.replace(/\bno-js\b/,'js')})(document.documentElement)</script>
    <meta name="generator" content="DokuWiki"/>
<meta name="theme-color" content="#008800"/>
<meta name="robots" content="n

In [20]:
import re
import requests
from bs4 import BeautifulSoup
from docx import Document

def read_docx(file_path, start_line, end_line):
    doc = Document(file_path)
    lines = []
    line_count = 0
    for paragraph in doc.paragraphs:
        if start_line <= line_count < end_line:
            lines.append(paragraph.text.strip())
        line_count += 1
    return lines

def parse_lines(lines):
    url_pattern = re.compile(r'https?://\S+')
    data = []
    current_main_category = None
    current_job = None
    jobs = {}

    for line in lines:
        if url_pattern.search(line):
            # Extract the URL
            url_match = url_pattern.search(line)
            url = url_match.group()
            
            # Extract the category (everything before the URL)
            category = line[:url_match.start()].strip()

            if current_job and current_job in jobs:
                jobs[current_job]['links'].append({
                    'url': url,
                    'category': category
                })
        elif line.lower().startswith("undefined"):
            # Skip lines starting with "undefined"
            continue
        else:
            if current_main_category is None:
                # Set the main category
                current_main_category = line
            elif current_job is None:
                # Set the job title (sub_category)
                current_job = line
                jobs[current_job] = {
                    'links': []
                }
            else:
                # Handle a new main category if a new line appears
                if line.strip() == "":
                    if current_main_category:
                        # Save the current main category and its jobs
                        data.append({
                            'main_category': current_main_category,
                            'jobs': jobs
                        })
                        # Reset for the next main category
                        current_main_category = None
                        jobs = {}
                        current_job = None
                else:
                    # If it's neither a URL nor an empty line, it might be a new job
                    if current_job:
                        # Make sure to add the current job to jobs before changing
                        current_job = line
                        jobs[current_job] = {
                            'links': []
                        }

    # Handle the last main category and jobs if they exist
    if current_main_category and jobs:
        data.append({
            'main_category': current_main_category,
            'jobs': jobs
        })

    return data

def format_content(links):
    """Format the content for the page including links."""
    content = "Here are some useful links:\n\n"
    for link in links:
        content += f"{link['category']}: {link['url']}\n"
    return content.strip()

def edit_page(career, content):
    """Fetch and update the page with the specified content."""
    edit_url = f"http://192.168.1.247/psyc_jobs/dokuwiki/doku.php?id=wiki:{career}&do=edit"
    response = requests.get(edit_url)

    if response.status_code != 200:
        print(f"Failed to fetch edit page for '{career}'. Status code: {response.status_code}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    sectok = soup.find('input', {'name': 'sectok'})
    if sectok:
        sectok = sectok['value']
    else:
        print(f"Failed to extract sectok for page: {career}")
        return

    edit_data = {
        'sectok': sectok,
        'id': f'wiki:{career}',
        'rev': '0',
        'date': '1724816599',
        'prefix': '.',
        'suffix': '',
        'changecheck': 'a119e534072584a0ea88cdea4664aecd',
        'target': 'section',
        'wikitext': content,
        'do[save]': '1'
    }

    response = requests.post(edit_url, data=edit_data)

    if response.status_code == 200:
        print(f"Page '{career}' edited successfully!")
    else:
        print(f"Failed to edit the page '{career}'. Status code: {response.status_code}")

def create_pages_from_docx(file_path, start_line, end_line):
    lines = read_docx(file_path, start_line, end_line)
    parsed_data = parse_lines(lines)

    for category in parsed_data:
        for job, details in category['jobs'].items():
            content = format_content(details['links'])
            edit_page(job, content)

# Specify the file path and line range
file_path = 'careers.docx'
start_line = 171  # Starting line (inclusive)
end_line = start_line + 3000  # Adjust the ending line as needed

create_pages_from_docx(file_path, start_line, end_line)


Page 'Advertising Sales Representative / Agent / Executive' edited successfully!
Page 'Assistant Bank Manager' edited successfully!
Page 'Claims Supervisor' edited successfully!
Page 'Customer Service Representative' edited successfully!
Page 'Customer Service Representative Supervisor' edited successfully!
Page 'Department Manager' edited successfully!
Page 'Financial Advisor' edited successfully!
Page 'Fund Raiser' edited successfully!
Page 'General Operations Manager' edited successfully!
Page 'Loan Counselor' edited successfully!
Page 'Loan Officer' edited successfully!
Page 'Loss Prevention Manager' edited successfully!
Page 'Management Analyst' edited successfully!
Page 'Market Research Analyst' edited successfully!
Page 'Medical and Health Services Manager' edited successfully!
Page 'Media Buyer' edited successfully!
Page 'Pharmaceutical Sales Representative' edited successfully!
Page 'Purchasing Agent' edited successfully!
Page 'Realtor or Real Estate Agent' edited successfully

In [21]:
import re
import requests
from bs4 import BeautifulSoup
from docx import Document

def read_docx(file_path, start_line, end_line):
    doc = Document(file_path)
    lines = []
    line_count = 0
    for paragraph in doc.paragraphs:
        if start_line <= line_count < end_line:
            lines.append(paragraph.text.strip())
        line_count += 1
    return lines

def parse_lines(lines):
    url_pattern = re.compile(r'https?://\S+')
    data = []
    current_main_category = None
    current_job = None
    jobs = {}

    for line in lines:
        if url_pattern.search(line):
            # Extract the URL
            url_match = url_pattern.search(line)
            url = url_match.group()
            
            # Extract the category (everything before the URL)
            category = line[:url_match.start()].strip()

            if current_job and current_job in jobs:
                jobs[current_job]['links'].append({
                    'url': url,
                    'category': category
                })
        elif line.lower().startswith("undefined"):
            # Skip lines starting with "undefined"
            continue
        else:
            if current_main_category is None:
                # Set the main category
                current_main_category = line
            elif current_job is None:
                # Set the job title (sub_category)
                current_job = line
                jobs[current_job] = {
                    'links': []
                }
            else:
                # Handle a new main category if a new line appears
                if line.strip() == "":
                    if current_main_category:
                        # Save the current main category and its jobs
                        data.append({
                            'main_category': current_main_category,
                            'jobs': jobs
                        })
                        # Reset for the next main category
                        current_main_category = None
                        jobs = {}
                        current_job = None
                else:
                    # If it's neither a URL nor an empty line, it might be a new job
                    if current_job:
                        # Make sure to add the current job to jobs before changing
                        current_job = line
                        jobs[current_job] = {
                            'links': []
                        }

    # Handle the last main category and jobs if they exist
    if current_main_category and jobs:
        data.append({
            'main_category': current_main_category,
            'jobs': jobs
        })

    return data

def format_content(links):
    """Format the content for the page including links."""
    content = "Here are some useful links:\n\n"
    for link in links:
        content += f"{link['category']}: {link['url']}\n"
    return content.strip()

def edit_page(career, content):
    """Fetch and update the page with the specified content."""
    edit_url = f"http://192.168.1.247/dokuwiki_test/doku.php?id=wiki:{career}&do=edit"
    response = requests.get(edit_url)

    if response.status_code != 200:
        print(f"Failed to fetch edit page for '{career}'. Status code: {response.status_code}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    sectok = soup.find('input', {'name': 'sectok'})
    if sectok:
        sectok = sectok['value']
    else:
        print(f"Failed to extract sectok for page: {career}")
        return

    edit_data = {
        'sectok': sectok,
        'id': f'wiki:{career}',
        'rev': '0',
        'date': '1724816599',
        'prefix': '.',
        'suffix': '',
        'changecheck': 'a119e534072584a0ea88cdea4664aecd',
        'target': 'section',
        'wikitext': content,
        'do[save]': '1'
    }

    response = requests.post(edit_url, data=edit_data)

    if response.status_code == 200:
        print(f"Page '{career}' edited successfully!")
    else:
        print(f"Failed to edit the page '{career}'. Status code: {response.status_code}")

def create_pages_from_docx(file_path, start_line, end_line):
    lines = read_docx(file_path, start_line, end_line)
    parsed_data = parse_lines(lines)

    for category in parsed_data:
        for job, details in category['jobs'].items():
            content = format_content(details['links'])
            edit_page(job, content)

# Specify the file path and line range
file_path = 'careers.docx'
start_line = 171  # Starting line (inclusive)
end_line = start_line + 3000  # Adjust the ending line as needed

create_pages_from_docx(file_path, start_line, end_line)


Page 'Advertising Sales Representative / Agent / Executive' edited successfully!
Page 'Assistant Bank Manager' edited successfully!
Page 'Claims Supervisor' edited successfully!
Page 'Customer Service Representative' edited successfully!
Page 'Customer Service Representative Supervisor' edited successfully!
Page 'Department Manager' edited successfully!
Page 'Financial Advisor' edited successfully!
Page 'Fund Raiser' edited successfully!
Page 'General Operations Manager' edited successfully!
Page 'Loan Counselor' edited successfully!
Page 'Loan Officer' edited successfully!
Page 'Loss Prevention Manager' edited successfully!
Page 'Management Analyst' edited successfully!
Page 'Market Research Analyst' edited successfully!
Page 'Medical and Health Services Manager' edited successfully!
Page 'Media Buyer' edited successfully!
Page 'Pharmaceutical Sales Representative' edited successfully!
Page 'Purchasing Agent' edited successfully!
Page 'Realtor or Real Estate Agent' edited successfully

In [23]:
import requests

# Define the base URL and authentication token
base_url = 'http://192.168.1.247/dokuwiki/doku.php'
auth_token = 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJkb2t1d2lraSIsInN1YiI6Im1sdzE5bWx3OTEiLCJpYXQiOjE3MjQ4MjEyNTR9.bpFN+b4PUC3B104AqQDw8xsKu7YegXv/nVEojUvuNUo='

# Headers with the token
headers = {
    'Authorization': f'Bearer {auth_token}'
}

# Example function to get page content
def get_page(page_id):
    url = f'{base_url}?id={page_id}'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        response.raise_for_status()

# Example function to save page content
def put_page(page_id, content):
    url = f'{base_url}?id={page_id}&do=save'
    data = {
        'wikitext': content,
        'summary': 'Updated via script'
    }
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        return 'Page updated successfully!'
    else:
        response.raise_for_status()

# Fetch page content
print(get_page('start'))

# Update page content
print(put_page('start', 'This is updated content'))


<!DOCTYPE html>
<html lang="en" dir="ltr" class="no-js">
<head>
    <meta charset="utf-8" />
    <title>start [Career Exploration Resource for Psychology Majors]</title>
    <script>(function(H){H.className=H.className.replace(/\bno-js\b/,'js')})(document.documentElement)</script>
    <meta name="generator" content="DokuWiki"/>
<meta name="theme-color" content="#008800"/>
<meta name="robots" content="noindex,follow"/>
<meta name="keywords" content="start"/>
<link rel="search" type="application/opensearchdescription+xml" href="/dokuwiki/lib/exe/opensearch.php" title="Career Exploration Resource for Psychology Majors"/>
<link rel="start" href="/dokuwiki/"/>
<link rel="contents" href="/dokuwiki/doku.php?id=start&amp;do=index" title="Sitemap"/>
<link rel="manifest" href="/dokuwiki/lib/exe/manifest.php"/>
<link rel="alternate" type="application/rss+xml" title="Recent Changes" href="/dokuwiki/feed.php"/>
<link rel="alternate" type="application/rss+xml" title="Current namespace" href="/dokuwi

## Create docuwiki page

In [25]:
import re
import requests
from docx import Document

# Define DokuWiki base URL and authentication token
base_url = 'http://192.168.1.247/dokuwiki/doku.php'
auth_token = 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJkb2t1d2lraSIsInN1YiI6Im1sdzE5bWx3OTEiLCJpYXQiOjE3MjQ4MjEyNTR9.bpFN+b4PUC3B104AqQDw8xsKu7YegXv/nVEojUvuNUo='

headers = {
    'Authorization': f'Bearer {auth_token}'
}

def read_docx(file_path, start_line, end_line):
    doc = Document(file_path)
    lines = []
    line_count = 0
    for paragraph in doc.paragraphs:
        if start_line <= line_count < end_line:
            lines.append(paragraph.text.strip())
        line_count += 1
    return lines

def parse_lines(lines):
    url_pattern = re.compile(r'https?://\S+')
    data = []
    current_main_category = None
    current_job = None
    jobs = {}

    for line in lines:
        if url_pattern.search(line):
            url_match = url_pattern.search(line)
            url = url_match.group()
            category = line[:url_match.start()].strip()

            if current_job and current_job in jobs:
                jobs[current_job]['links'].append({
                    'url': url,
                    'category': category
                })
        elif line.lower().startswith("undefined"):
            continue
        else:
            if current_main_category is None:
                current_main_category = line
            elif current_job is None:
                current_job = line
                jobs[current_job] = {
                    'links': []
                }
            else:
                if line.strip() == "":
                    if current_main_category:
                        data.append({
                            'main_category': current_main_category,
                            'jobs': jobs
                        })
                        current_main_category = None
                        jobs = {}
                        current_job = None
                else:
                    if current_job:
                        current_job = line
                        jobs[current_job] = {
                            'links': []
                        }

    if current_main_category and jobs:
        data.append({
            'main_category': current_main_category,
            'jobs': jobs
        })

    return data

def get_page(page_id):
    url = f'{base_url}?id={page_id}'
    print(f'GET {url}')
    response = requests.get(url, headers=headers)
    print(f'Status Code: {response.status_code}')
    if response.status_code == 200:
        return response.text
    else:
        response.raise_for_status()

def put_page(page_id, content):
    url = f'{base_url}?id={page_id}&do=save'
    data = {
        'wikitext': content,
        'summary': 'Updated via script'
    }
    print(f'POST {url} with data: {data}')
    response = requests.post(url, headers=headers, data=data)
    print(f'Status Code: {response.status_code}')
    if response.status_code == 200:
        return 'Page updated successfully!'
    else:
        response.raise_for_status()

def create_or_update_page(title, content):
    try:
        existing_content = get_page(title)
        print(f'Updating page: {title}')
        print(put_page(title, content))
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f'Creating page: {title}')
            content = f"Creating new page: {title}\n\n{content}"
            print(put_page(title, content))
        else:
            raise

def main(file_path, start_line, end_line):
    lines = read_docx(file_path, start_line, end_line)
    data = parse_lines(lines)

    for entry in data:
        main_category = entry['main_category']
        for job_title, job_data in entry['jobs'].items():
            content = f"## {job_title}\n\n"
            for link in job_data['links']:
                content += f"- [Link]({link['url']}) - {link['category']}\n"
            create_or_update_page(job_title, content)

file_path = 'careers.docx'
start_line = 171
end_line = start_line + 3000

main(file_path, start_line, end_line)


GET http://192.168.1.247/dokuwiki/doku.php?id=Advertising Sales Representative / Agent / Executive
Status Code: 200
Updating page: Advertising Sales Representative / Agent / Executive
POST http://192.168.1.247/dokuwiki/doku.php?id=Advertising Sales Representative / Agent / Executive&do=save with data: {'wikitext': '## Advertising Sales Representative / Agent / Executive\n\n- [Link](http://online.onetcenter.org/link/summary/41-3011.00) - O*NET:\n- [Link](http://www.bls.gov/ooh/sales/advertising-sales-agents.htm) - OOH:\n- [Link](http://www.ehow.com/about_6123748_job-description-advertising-sales-rep.html) - Wild Card:\n- [Link](https://www.drkit.org/account-executive/) - Video:\n- [Link](http://www.youtube.com/watch?v=YxH7xG8WzvY) - Video:\n- [Link](https://www.youtube.com/watch?v=ovjmCc1R6EU) - Video:\n', 'summary': 'Updated via script'}
Status Code: 200
Page updated successfully!
GET http://192.168.1.247/dokuwiki/doku.php?id=Assistant Bank Manager
Status Code: 200
Updating page: Assis

KeyboardInterrupt: 

In [31]:
from dokuwiki import DokuWiki

# Define DokuWiki base URL and credentials
base_url = 'http://192.168.1.247/dokuwiki/lib/exe/xmlrpc.php'
username = 'mlw19bmlw91'
password = 'Mlw0112358!'

try:
    # Initialize DokuWiki instance
    wiki = DokuWiki(url=base_url, user=username, password=password)
    version = wiki.version
    print(f'DokuWiki version: {version}')
except Exception as e:
    print(f'Error: {e}')


Error: <Fault -32605: 'XML-RPC server not enabled.'>
